In [7]:
import cv2
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def imgdir(base, image_files=[]):
    for img in os.listdir(base):
        image_file = os.path.join(base, img)
        image_files.append(image_file)
    return image_files

def process_image(filename, output_dir):
    
    img = cv2.imread(filename)  # Read the image

    # Create a binary mask for white pixels (all RGB channels > 200)
    mask = np.all(img > 200, axis=2).astype(np.uint8) * 255

    # Construct the output file path
    output_filename = os.path.join(
        output_dir, os.path.basename(filename).replace('.jpg', '.png')
    )

    # Save the mask in the output directory
    cv2.imwrite(output_filename, mask)
    
    return np.sum(mask > 200)  # Return the count of white pixels

def main(base, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get all image file paths from the base directory
    image_files = imgdir(base)

    # Process images in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda f: process_image(f, output_dir), image_files))

    # Compute the total white pixels across all images
    total_white_pixels = sum(results)
    return total_white_pixels

# Specify the input and output directories
base = r"D:\Test_result\data"
output_dir = r"D:\Test_result\masks"

# Run the main function
total_pixels = main(base, output_dir)
print(f"Total white pixels: {total_pixels}")


Total white pixels: 23157
